In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

import blib
blib.useTheme("dark")

import src.radarkit.map as map

In [ ]:
r_earth = 6378.0
origin = (-97.44, 35.22)
extent = (-160, 160, -90, 90)
# extent = (-1600, 1600, -900, 900)

In [ ]:
# paths = map.subsetPaths('state', origin, extent)
paths_county = map.subsetPaths('county', origin, extent)
paths_highway = map.subsetPaths('highway', origin, extent)

In [ ]:
fig = plt.figure(dpi=72)
ax = fig.add_axes([0, 0, 1, 1])
for p in paths_county:
    ax.add_line(matplotlib.lines.Line2D(p[:, 0], p[:, 1], linewidth=0.8, color=map.colors.county))
for p in paths_highway:
    ax.add_line(matplotlib.lines.Line2D(p[:, 0], p[:, 1], color=map.colors.highway))
ax.axis('equal')
ax.set(xlim=extent[:2], ylim=extent[-2:])

In [ ]:
plt.close()